In [4]:
import random
import math
import os

class VRPGenerator:
    def __init__(self):
        self.random_seed = 42
        random.seed(self.random_seed)
    
    def euclidean_distance(self, p1, p2):
        """计算两点间的欧几里得距离"""
        return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)
    
    def generate_coordinates(self, n_nodes, coord_range=(20, 80)):
        """生成节点坐标"""
        coordinates = []
        for i in range(n_nodes):
            x = random.randint(coord_range[0], coord_range[1])
            y = random.randint(coord_range[0], coord_range[1])
            coordinates.append((x, y))
        return coordinates
    
    def generate_demands(self, n_nodes, demand_range=(5, 35)):
        """生成需求量，第一个节点（仓库）需求为0"""
        demands = [0]  # 仓库需求为0
        for i in range(1, n_nodes):
            demand = random.randint(demand_range[0], demand_range[1])
            demands.append(demand)
        return demands
    
    def estimate_optimal_value(self, coordinates, demands, n_trucks):
        """粗略估算最优值（基于距离和需求分布）"""
        depot = coordinates[0]
        total_distance = 0
        
        # 计算所有客户到仓库的距离
        customer_distances = []
        for i in range(1, len(coordinates)):
            dist = self.euclidean_distance(depot, coordinates[i])
            customer_distances.append(dist)
        
        # 简单估算：平均每车服务的客户数 * 平均往返距离
        avg_customers_per_truck = (len(coordinates) - 1) / n_trucks
        avg_distance = sum(customer_distances) / len(customer_distances)
        
        # 估算的最优值（包含一些随机因子）
        estimated = int(avg_distance * 2 * n_trucks * (1 + random.uniform(0.1, 0.3)))
        return max(estimated, 100)  # 确保最小值
    
    def generate_vrp_instance(self, name, n_nodes, n_trucks, capacity, coord_range=(20, 80), demand_range=(5, 35)):
        """生成一个VRP实例"""
        coordinates = self.generate_coordinates(n_nodes, coord_range)
        demands = self.generate_demands(n_nodes, demand_range)
        optimal_value = self.estimate_optimal_value(coordinates, demands, n_trucks)
        
        return {
            'name': name,
            'n_nodes': n_nodes,
            'n_trucks': n_trucks,
            'capacity': capacity,
            'optimal_value': optimal_value,
            'coordinates': coordinates,
            'demands': demands
        }
    
    def write_vrp_file(self, instance, filename):
        """将VRP实例写入文件"""
        with open(filename, 'w') as f:
            f.write(f"NAME : {instance['name']}\n")
            f.write(f"COMMENT : (Generated instance, No of trucks: {instance['n_trucks']}, Optimal value: {instance['optimal_value']})\n")
            f.write(f"TYPE : CVRP\n")
            f.write(f"DIMENSION : {instance['n_nodes']}\n")
            f.write(f"EDGE_WEIGHT_TYPE : EUC_2D\n")
            f.write(f"CAPACITY : {instance['capacity']}\n")
            f.write(f"NODE_COORD_SECTION\n")
            
            # 写入坐标
            for i, (x, y) in enumerate(instance['coordinates'], 1):
                f.write(f"{i} {x} {y}\n")
            
            f.write(f"DEMAND_SECTION\n")
            
            # 写入需求
            for i, demand in enumerate(instance['demands'], 1):
                f.write(f"{i} {demand}\n")
            
            f.write(f"DEPOT_SECTION\n")
            f.write(f" 1\n")
            f.write(f" -1\n")
            f.write(f"EOF\n")
    
    def generate_dataset(self, output_dir="generated_vrp"):
        """生成10个VRP数据集"""
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        # 定义10个小规模实例配置（节点数8-15）
        configs = [
            {"name": "Gen-n8-k2.vrp", "n_nodes": 8, "n_trucks": 2, "capacity": 50, "coord_range": (20, 70), "demand_range": (8, 25)},
            {"name": "Gen-n10-k3.vrp", "n_nodes": 10, "n_trucks": 3, "capacity": 40, "coord_range": (25, 75), "demand_range": (6, 20)},
            {"name": "Gen-n12-k3.vrp", "n_nodes": 12, "n_trucks": 3, "capacity": 45, "coord_range": (15, 80), "demand_range": (5, 18)},
            {"name": "Gen-n9-k2.vrp", "n_nodes": 9, "n_trucks": 2, "capacity": 60, "coord_range": (30, 90), "demand_range": (10, 30)},
            {"name": "Gen-n15-k4.vrp", "n_nodes": 15, "n_trucks": 4, "capacity": 35, "coord_range": (20, 85), "demand_range": (4, 15)},
            {"name": "Gen-n11-k3.vrp", "n_nodes": 11, "n_trucks": 3, "capacity": 50, "coord_range": (25, 65), "demand_range": (8, 22)},
            {"name": "Gen-n13-k3.vrp", "n_nodes": 13, "n_trucks": 3, "capacity": 55, "coord_range": (10, 70), "demand_range": (6, 25)},
            {"name": "Gen-n14-k4.vrp", "n_nodes": 14, "n_trucks": 4, "capacity": 40, "coord_range": (35, 80), "demand_range": (9, 20)},
            {"name": "Gen-n8-k3.vrp", "n_nodes": 8, "n_trucks": 3, "capacity": 30, "coord_range": (15, 75), "demand_range": (5, 18)},
            {"name": "Gen-n12-k2.vrp", "n_nodes": 12, "n_trucks": 2, "capacity": 70, "coord_range": (20, 90), "demand_range": (12, 28)}
        ]
        
        instances = []
        
        for i, config in enumerate(configs, 1):
            # 为每个实例设置不同的随机种子以确保多样性
            random.seed(self.random_seed + i * 10)
            
            instance = self.generate_vrp_instance(**config)
            instances.append(instance)
            
            filename = os.path.join(output_dir, f"{config['name']}.txt")
            self.write_vrp_file(instance, filename)
            
            print(f"生成实例 {i}: {config['name']}")
            print(f"  节点数: {instance['n_nodes']}, 卡车数: {instance['n_trucks']}")
            print(f"  容量: {instance['capacity']}, 估算最优值: {instance['optimal_value']}")
            print(f"  文件保存至: {filename}")
            print()
        
        return instances

# 使用示例
if __name__ == "__main__":
    generator = VRPGenerator()
    instances = generator.generate_dataset()
    
    print(f"成功生成 {len(instances)} 个VRP数据集!")
    print("所有文件已保存到 'generated_vrp' 目录中")
    
    # 验证生成的第一个实例
    print("\n=== 第一个生成实例的详细信息 ===")
    first_instance = instances[0]
    print(f"名称: {first_instance['name']}")
    print(f"坐标前5个: {first_instance['coordinates'][:5]}")
    print(f"需求前5个: {first_instance['demands'][:5]}")
    print(f"总需求量: {sum(first_instance['demands'])}")
    print(f"平均每车承载: {sum(first_instance['demands']) / first_instance['n_trucks']:.1f}")

生成实例 1: Gen-n8-k2.vrp
  节点数: 8, 卡车数: 2
  容量: 50, 估算最优值: 100
  文件保存至: generated_vrp\Gen-n8-k2.vrp.txt

生成实例 2: Gen-n10-k3.vrp
  节点数: 10, 卡车数: 3
  容量: 40, 估算最优值: 156
  文件保存至: generated_vrp\Gen-n10-k3.vrp.txt

生成实例 3: Gen-n12-k3.vrp
  节点数: 12, 卡车数: 3
  容量: 45, 估算最优值: 272
  文件保存至: generated_vrp\Gen-n12-k3.vrp.txt

生成实例 4: Gen-n9-k2.vrp
  节点数: 9, 卡车数: 2
  容量: 60, 估算最优值: 163
  文件保存至: generated_vrp\Gen-n9-k2.vrp.txt

生成实例 5: Gen-n15-k4.vrp
  节点数: 15, 卡车数: 4
  容量: 35, 估算最优值: 291
  文件保存至: generated_vrp\Gen-n15-k4.vrp.txt

生成实例 6: Gen-n11-k3.vrp
  节点数: 11, 卡车数: 3
  容量: 50, 估算最优值: 190
  文件保存至: generated_vrp\Gen-n11-k3.vrp.txt

生成实例 7: Gen-n13-k3.vrp
  节点数: 13, 卡车数: 3
  容量: 55, 估算最优值: 192
  文件保存至: generated_vrp\Gen-n13-k3.vrp.txt

生成实例 8: Gen-n14-k4.vrp
  节点数: 14, 卡车数: 4
  容量: 40, 估算最优值: 213
  文件保存至: generated_vrp\Gen-n14-k4.vrp.txt

生成实例 9: Gen-n8-k3.vrp
  节点数: 8, 卡车数: 3
  容量: 30, 估算最优值: 196
  文件保存至: generated_vrp\Gen-n8-k3.vrp.txt

生成实例 10: Gen-n12-k2.vrp
  节点数: 12, 卡车数: 2
  容量: 70, 估算最优值: 150
 